In [2]:
import  torch
import  numpy as np
import  matplotlib.pyplot as plt
from    keras import datasets
import  import_ipynb

from RegretBound import opt_param
from data import MNIST_data
from model import OFL_Model
from utils import sigma_diff

K = 100         # Number of clients
D = 34826       # number of parameters (MNIST)
P = 1           # Com. overhead reduction rate from FedOGD
L = 10          # Local training time (Batch Size)

s, _, b, p = opt_param(P, D, show_result=False)
p2 = p * L
print(s, b, p, p2)

1 34826 0.9411764705882353 9.411764705882353


In [3]:
x_train, y_train, input_size = MNIST_data(iid=True, shuffle=False)
x_train2, y_train2, input_size = MNIST_data(iid=False, shuffle=False)
task = 'clf'

Model_list = []

Model_list.append(OFL_Model('FedOGD_sigma_iid', task, K, [False, 0, 0], 1, L, input_size))
Model_list.append(OFL_Model('FedOGD_sigma_niid', task, K, [False, 0, 0], 1, L, input_size))
# Model_list.append(OFL_Model('OFedAvg', task, K, [False, 0, 0], P, 1, input_size))
# Model_list.append(OFL_Model('FedOMD', task, K, [False, 0, 0], 1, 10, input_size))
# Model_list.append(OFL_Model('OFedIQ', task, K, [True, s, b], p, 1, input_size))

2022-12-29 16:27:53.015692: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
i_max = 375
iter_max = 2
print(iter_max, i_max)

model = Model_list[0]
for iter in range(iter_max):
    for i in range(i_max):
        model.train(x_train[K*i : K*(i+1)], y_train[K*i : K*(i+1)], ((i_max * iter) + (i+1)) % model.L)
    print(iter, end =',')

model = Model_list[1]
for iter in range(iter_max):
    for i in range(i_max):
        model.train(x_train2[K*i : K*(i+1)], y_train2[K*i : K*(i+1)], ((i_max * iter) + (i+1)) % model.L)
    print(iter, end =',')

result_list = []
for model in Model_list:
    result_list.append(model.pull_result())

2 375


In [ ]:
import pickle

code = input("Code: ")

for i, model in enumerate(Model_list):
    name = model.name
    with open(f"./result/{task}_{name}_{code}.pkl","wb") as f:
        pickle.dump(result_list[i], f)

In [ ]:
import pickle 

with open(f"./result/clf_FedOGD_sigma_iid.pkl", "rb") as f:
    r1 = pickle.load(f)
with open(f"./result/clf_FedOGD_sigma_niid.pkl", "rb") as f:
    r2 = pickle.load(f)

print(r1[250], r1[500])
print(r2[250], r2[500])